In [2]:
from bs4 import BeautifulSoup
import pandas as pd

import numpy as np
import json # library to handle JSON files

#!conda install -c conda-forge geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# # Introduction
Singapore is a international vibrant country city in Asia. Singapore is the world's most competitive economy, according to an updated global league table. It scored 84.8 out of a possible 100, beating the United States to the top spot in the ranking of 141 economies.The US, which topped the charts last year, scored 83.7. Thanks to its strategic location, reliable government and business-friendly environement, Singapore become top choice for global business enitities to put their APAC head quarters in. Expariates pick Singapore as their most wanted place for their next move because it's safe, bilingual and multinationalized. One possible and real life question firstly come up when Expariate relocate to Singapore would be,where should I live, or where should I rent my house if the accomodation is not arranged by company which should be most of the cases. Most company provides relocation package and allowance to let you choose yourself the place you want to stay. Singapore is a small island but contains 28 districts, which further splits into different areas. It's indeed a headache for those who come to Singapore for first time to choose which neighborhood is best for them. 
In this project, I will go through the data from different districts and areas using four square api to explore these neighborhood. 

# # Data
To simplify the analysis, assumption that the expatriates mostly work in CBD area or we called Core Central Region (CCR) in Singapore has been made.Depends on Expatriate profile, he or she might come to Singapore alone or with family, so a two-bed room condo unit or one-bedroom unit might be in favour to them probably. With a budget of 3000SGD, an expatriates could get a one-bedroom or just a studio in CCR region, while in Rest of Central Region (RCR), he/she could rent a two-bedroom condo unit. The commute time from RCR to CBD is within 30min by public transport or less than 20min by car, which should be acceptable. 
To start with RCR areas in Singapore, I googled "Singapore RCR" and retrieved a table of areas in each of the RCR districts.

In [3]:
source = requests.get('https://www.iproperty.com.sg/news/what-do-ccr-rcr-and-ocr-mean-in-singapore-property/').text
soup = BeautifulSoup(source, 'lxml')
table = soup.findAll('table')[1]
#print(table.prettify())
sg_df = pd.read_html(str(table))[0]
sg_df.columns = sg_df.iloc[0]
sg_df = sg_df[1:]
sg_df

,District,Area
1,3,"Alexandra Road, Tiong Bahru, Queenstown"
2,4,"Keppel, Mount Faber, Telok Blangah"
3,5,"Buona Vista, Dover, Pasir Panjang, West Coast"
4,7,"Beach Road, Bencoolen Road, Bugis, Rochor"
5,8,"Little India, Farrer Park, Serangoon Road"
6,12,"Balestier, Moulmein, Novena, Toa Payoh"
7,13,"Potong Pasir, Macpherson"
8,14,"Eunos, Geylang, Kembangan, Paya Lebar"
9,15,"Katong, Marine Parade, Siglap, Tanjong Rhu"
10,20,"Ang Mo Kio, Bishan, Braddell Road, Thomson"


These general areas will be used to explore the most popular venues through FourSquare API. Let's first seperate the areas into columns, and sort them in the order of district they belonged to repectively.

In [7]:
rcr_sg_df = sg_df.Area.str.split(',').apply(pd.Series).merge(sg_df.District.astype(int),left_index=True, right_index=True).melt(id_vars='District').dropna().drop(['variable'],axis=1).sort_values(by=['District'], ascending=True).rename(columns={"value":"Area"}).reset_index(drop=True)
rcr_sg_df.Area = rcr_sg_df.Area + ', Singapore'
rcr_sg_df

,District,Area
0,3,"Alexandra Road, Singapore"
1,3,"Queenstown, Singapore"
2,3,"Tiong Bahru, Singapore"
3,4,"Keppel, Singapore"
4,4,"Telok Blangah, Singapore"
5,4,"Mount Faber, Singapore"
6,5,"Pasir Panjang, Singapore"
7,5,"West Coast, Singapore"
8,5,"Dover, Singapore"
9,5,"Buona Vista, Singapore"


We will use geopy package to aquire the geographic coordinates (Lagtitude and longtitude) of each area. To save time and avoid "service timed out" issue seen from geolocator, I export the coordinates to an csv file for future extraction.

In [5]:
address = 'Pasir Panjang, Singapore'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.2888336, 103.7756574.


In [8]:
rcr_sg_df['Latitude']=rcr_sg_df['Area'].apply(geolocator.geocode).apply(lambda x: (x.latitude))

In [9]:
rcr_sg_df['Longitude']=rcr_sg_df['Area'].apply(geolocator.geocode).apply(lambda x: (x.longitude))
rcr_sg_df

,District,Area,Latitude,Longitude
0,3,"Alexandra Road, Singapore",1.278088,103.802975
1,3,"Queenstown, Singapore",1.294623,103.806045
2,3,"Tiong Bahru, Singapore",1.286197,103.825765
3,4,"Keppel, Singapore",1.274836,103.829539
4,4,"Telok Blangah, Singapore",1.270586,103.809863
5,4,"Mount Faber, Singapore",1.271152,103.819459
6,5,"Pasir Panjang, Singapore",1.288834,103.775657
7,5,"West Coast, Singapore",1.307230,103.766728
8,5,"Dover, Singapore",1.311421,103.778627
9,5,"Buona Vista, Singapore",1.292460,103.787773


In [10]:
rcr_sg_df.to_csv(r'Singapore_RCR_Coordinates.csv')

Plot on the map to see the areas spreading in the whole island. Noted that the downtown of Singapore is not at the centre but rather South of island. 

In [19]:
map_singapore = folium.Map(location=[latitude, longitude], zoom_start=12)
folium.CircleMarker(
        [1.277685,103.8463559],
        radius=8,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186ff',
        fill_opacity=0.8,
        parse_html=True).add_to(map_singapore)

# add markers to map
for lat, lng, label in zip(rcr_sg_df['Latitude'], rcr_sg_df['Longitude'], rcr_sg_df['Area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_singapore) 


map_singapore